# Uso básico de Pyparsing

In [1]:
import pyparsing as pp

In [2]:
#
# Ejemplo: Hola mundo
#

#
# Define la gramática como:
# 
# greet -> string ',' string '!'
#
greet = pp.Word(pp.alphas) + "," + pp.Word(pp.alphas) + "!"


# procesa saludos en varios idiomas:
for greeting_str in [
            "Hello, World!",
            "Bonjour, Monde!",
            "Hola, Mundo!",
            "Hallo, Welt!",
        ]:
    greeting = greet.parseString(greeting_str)
    print(greeting)

['Hello', ',', 'World', '!']
['Bonjour', ',', 'Monde', '!']
['Hola', ',', 'Mundo', '!']
['Hallo', ',', 'Welt', '!']


In [3]:
#
# Ejemplo: Pasing de una dirección IP y un número telefonico
# en formato US:
#
#    111.222.333.444(123)456-7890
#    131.322.393.458(599)353-7800
#

ipField = pp.Word(pp.nums, max=3)
ipAddr = pp.Combine(ipField + "." + ipField + "." + ipField + "." + ipField)
phoneNum = pp.Combine(
    "("
    + pp.Word(pp.nums, exact=3)
    + ")"
    + pp.Word(pp.nums, exact=3)
    + "-"
    + pp.Word(pp.nums, exact=4)
)
userdata = ipAddr + phoneNum

for text in [
    "111.222.333.444(123)456-7890",
    "131.322.393.458(599)353-7800",
]:
    parsed_text = userdata.parseString(text)
    print(parsed_text)

['111.222.333.444', '(123)456-7890']
['131.322.393.458', '(599)353-7800']


In [4]:
# 
# Ejemplo: gramática para asignaciones en lenguajes
# de programación.
#
# Ejemplo:
#
#    a = 10
#    a_2=100
#    pi=3.14159
#    goldenRatio = 1.61803
#    E = mc2
#
identifier = pp.Word(pp.alphas, pp.alphanums+'_')
number = pp.Word(pp.nums+".")
assignmentExpr = identifier + "=" + (identifier | number)

for text in [
    "a = 10",
    "a_2=100",
    "pi=3.14159",
    "goldenRatio = 1.61803",
    "E = mc2",
]:
    parsed_text = assignmentExpr.parseString(text)
    print(parsed_text)

['a', '=', '10']
['a_2', '=', '100']
['pi', '=', '3.14159']
['goldenRatio', '=', '1.61803']
['E', '=', 'mc2']


In [5]:
#
# Asignación de nombres a partes de la expresión
# usando setResultName()
#
identifier = pp.Word(pp.alphas, pp.alphanums + "_")
number = pp.Word(pp.nums + ".")

## define los nombres de las partes
assignmentExpr = (
    identifier.setResultsName("lhs") + "=" + (identifier | number).setResultsName("rhs")
)

## parser
assignmentTokens = assignmentExpr.parseString("pi=3.14159")

## imprime las componentes usando los nombres asignados
print(assignmentTokens.rhs, "is assigned to", assignmentTokens.lhs)

3.14159 is assigned to pi


In [6]:
#
# Ejemplo de una gramática más compleja para
# parsear los siguientes textos:
#
#    Hello, World!
#    Hi, Mom!
#    Good morning, Miss Crabtree!
#    Yo, Adrian!
#    Whattup, G?
#    How's it goin', Dude?
#    Hey, Jude!
#    Goodbye, Mr. Chips!
#
word = pp.Word(pp.alphas+"'.")
salutation = pp.OneOrMore(word)
comma = pp.Literal(",")
greetee = pp.OneOrMore(word)
endpunc = pp.oneOf("! ?")
greeting = salutation + comma + greetee + endpunc

for text in [
    "Hello, World!",
    "Hi, Mom!",
    "Good morning, Miss Crabtree!",
    "Yo, Adrian!",
    "Whattup, G?",
    "How's it goin', Dude?",
    "Hey, Jude!",
    "Goodbye, Mr. Chips!",
]:
    parsed_text = greeting.parseString(text)
    print(parsed_text)

['Hello', ',', 'World', '!']
['Hi', ',', 'Mom', '!']
['Good', 'morning', ',', 'Miss', 'Crabtree', '!']
['Yo', ',', 'Adrian', '!']
['Whattup', ',', 'G', '?']
["How's", 'it', "goin'", ',', 'Dude', '?']
['Hey', ',', 'Jude', '!']
['Goodbye', ',', 'Mr.', 'Chips', '!']


In [7]:
#
# Extracción de la parte del saludo
#
for t in [
    "Hello, World!",
    "Hi, Mom!",
    "Good morning, Miss Crabtree!",
    "Yo, Adrian!",
    "Whattup, G?",
    "How's it goin', Dude?",
    "Hey, Jude!",
    "Goodbye, Mr. Chips!",
]:
    results = greeting.parseString(t)
    salutation = []
    for token in results:
        if token == ",": break
        salutation.append(token)
    print(salutation)

['Hello']
['Hi']
['Good', 'morning']
['Yo']
['Whattup']
["How's", 'it', "goin'"]
['Hey']
['Goodbye']


In [8]:
#
# Adición de grupos usando Group()
#
word = pp.Word(pp.alphas+"'.")
salutation = pp.Group(pp.OneOrMore(word))  ## <- regla modificada
comma = pp.Literal(",")
greetee = pp.Group( pp.OneOrMore(word) )  ## <- regla modificada
endpunc = pp.oneOf("! ?")
greeting = salutation + comma + greetee + endpunc

for t in [
    "Hello, World!",
    "Hi, Mom!",
    "Good morning, Miss Crabtree!",
    "Yo, Adrian!",
    "Whattup, G?",
    "How's it goin', Dude?",
    "Hey, Jude!",
    "Goodbye, Mr. Chips!",
]:
    parsed_text = greeting.parseString(t)
    print(parsed_text)

[['Hello'], ',', ['World'], '!']
[['Hi'], ',', ['Mom'], '!']
[['Good', 'morning'], ',', ['Miss', 'Crabtree'], '!']
[['Yo'], ',', ['Adrian'], '!']
[['Whattup'], ',', ['G'], '?']
[["How's", 'it', "goin'"], ',', ['Dude'], '?']
[['Hey'], ',', ['Jude'], '!']
[['Goodbye'], ',', ['Mr.', 'Chips'], '!']


In [9]:
#
# Es posible asignar cada parte a una 
# variable para su uso posterior
#
for t in [
    "Hello, World!",
    "Hi, Mom!",
    "Good morning, Miss Crabtree!",
    "Yo, Adrian!",
    "Whattup, G?",
    "How's it goin', Dude?",
    "Hey, Jude!",
    "Goodbye, Mr. Chips!",
]:
    salutation, _, greetee, endpunc = greeting.parseString(t)
    print(salutation, greetee, endpunc)

['Hello'] ['World'] !
['Hi'] ['Mom'] !
['Good', 'morning'] ['Miss', 'Crabtree'] !
['Yo'] ['Adrian'] !
['Whattup'] ['G'] ?
["How's", 'it', "goin'"] ['Dude'] ?
['Hey'] ['Jude'] !
['Goodbye'] ['Mr.', 'Chips'] !


In [10]:
#
# Supresión de elementos con Suppress
#
word = pp.Word(pp.alphas+"'.")
salutation = pp.Group(pp.OneOrMore(word))  ## <- regla modificada
comma = pp.Suppress(pp.Literal(","))
greetee = pp.Group( pp.OneOrMore(word) )  ## <- regla modificada
endpunc = pp.oneOf("! ?")
greeting = salutation + comma + greetee + endpunc


for t in [
    "Hello, World!",
    "Hi, Mom!",
    "Good morning, Miss Crabtree!",
    "Yo, Adrian!",
    "Whattup, G?",
    "How's it goin', Dude?",
    "Hey, Jude!",
    "Goodbye, Mr. Chips!",
]:
    salutation, greetee, endpunc = greeting.parseString(t)
    print(salutation, greetee, endpunc)

['Hello'] ['World'] !
['Hi'] ['Mom'] !
['Good', 'morning'] ['Miss', 'Crabtree'] !
['Yo'] ['Adrian'] !
['Whattup'] ['G'] ?
["How's", 'it', "goin'"] ['Dude'] ?
['Hey'] ['Jude'] !
['Goodbye'] ['Mr.', 'Chips'] !


In [11]:
#
# Separación en partes usando listas
#
salutes = []
greetees = []

for t in [
    "Hello, World!",
    "Hi, Mom!",
    "Good morning, Miss Crabtree!",
    "Yo, Adrian!",
    "Whattup, G?",
    "How's it goin', Dude?",
    "Hey, Jude!",
    "Goodbye, Mr. Chips!",
]:
    salutation, greetee, endpunc = greeting.parseString(t)
    salutes.append( ( " ".join(salutation), endpunc) )
    greetees.append( " ".join(greetee) )
    
print(salutes)
print('---')
print(greetees)

[('Hello', '!'), ('Hi', '!'), ('Good morning', '!'), ('Yo', '!'), ('Whattup', '?'), ("How's it goin'", '?'), ('Hey', '!'), ('Goodbye', '!')]
---
['World', 'Mom', 'Miss Crabtree', 'Adrian', 'G', 'Dude', 'Jude', 'Mr. Chips']


In [12]:
#
# Generación de cadenas aleatorias
#
import random

for i in range(20):
    salute = random.choice( salutes )
    greetee = random.choice( greetees )
    print("{:s}, {:s}{:s}".format( salute[0], greetee, salute[1] ))

Goodbye, Dude!
How's it goin', Mom?
How's it goin', Miss Crabtree?
How's it goin', Mr. Chips?
Goodbye, Mr. Chips!
Whattup, Adrian?
Hey, Mr. Chips!
Yo, Jude!
How's it goin', Mr. Chips?
Hi, Jude!
Hello, Mom!
Hey, Jude!
Yo, Mr. Chips!
Goodbye, Adrian!
How's it goin', G?
Hi, Mr. Chips!
Hey, Mr. Chips!
Whattup, World?
Whattup, Mom?
Goodbye, Dude!


In [13]:
#
# Otro ejemplo de frases aleatorias
#
for i in range(20):
    print(
        '{:s}, say "{:s}" to {:s}.'.format(
            random.choice(greetees),
            "".join(random.choice(salutes)),
            random.choice(greetees),
        )
    )

Adrian, say "Yo!" to Miss Crabtree.
Mom, say "Goodbye!" to World.
World, say "Hello!" to Miss Crabtree.
Miss Crabtree, say "Hi!" to Dude.
Miss Crabtree, say "Whattup?" to G.
G, say "Good morning!" to G.
World, say "Yo!" to Jude.
World, say "Goodbye!" to Mom.
World, say "Hello!" to Mr. Chips.
Adrian, say "Hey!" to World.
World, say "Yo!" to Miss Crabtree.
Miss Crabtree, say "Hey!" to G.
Miss Crabtree, say "Hello!" to Adrian.
G, say "Whattup?" to Mr. Chips.
Miss Crabtree, say "Whattup?" to G.
Jude, say "Whattup?" to G.
Miss Crabtree, say "Yo!" to Mr. Chips.
Jude, say "Hello!" to Adrian.
Jude, say "Good morning!" to World.
Mom, say "Yo!" to Adrian.
